In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Run cell to increase font sizes. Usefull when saving plots
SMALL_SIZE = 20
MEDIUM_SIZE = 24
BIGGER_SIZE = 28

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams["figure.figsize"] = (12,9)

In [3]:
# def rhs(w, t, parameters):
#    M, A = w
#    L, Vmax, n, Km, k1, k2, k3, k4, k5 = parameters

#    return [
#        k1 + k2 * (1 / (1 + A**n)) - k3 * M,
#        k4 * M * L - k5 * A - Vmax  * M * A * / (Km + A)
#    ]

def dMdt(M, A, L, Vmax=1, n=5, Km=2, k1=0.05, k2=1, k3=1, k4=1, k5=0.2):
    return k1 + k2 * (1 - (1/(1 + A**n))) - k3 * M

def dAdt(M, A, L, Vmax=1, n=5, Km=2, k1=0.05, k2=1, k3=1, k4=1, k5=0.2):
    return k4 * M * L - k5 * A - Vmax* M * A / (Km + A)

def M_nullcline(M, A, L, Vmax=1, n=5, Km=2, k1=0.05, k2=1, k3=1, k4=1, k5=0.2):

    return (1 / (1 - (k3 * M - k1) / k2) - 1 ) ** (1/n)

def A_nullcline(M, A, L, Vmax=1, n=5, Km=2, k1=0.05, k2=1, k3=1, k4=1, k5=0.2):

    a = -k5
    b = k4 * M * L - k5 * Km - Vmax * M
    c = k4 * M * L * Km

    d = b ** 2 - 4 * a * c
    return (-b - d ** 0.5) / (2 * a), (-b + d ** 0.5) / (2 * a)


L = 3
As = np.arange(0, 3, 0.01)
Ms = np.arange(0, 2, 0.01)

x_velocities, y_velocities = [], []
for A in As:

    x_temp, y_temp = [], []
    
    for M in Ms:

        x_temp.append(dMdt(M, A, L))
        y_temp.append(dAdt(M, A, L))

    x_velocities.append(x_temp)
    y_velocities.append(y_temp)

m_nulls = M_nullcline(Ms, As, L)
a_nulls_min, a_nulls_plus = A_nullcline(Ms, As, L)

<ipython-input-3-f0486a886956>:18: RuntimeWarning: divide by zero encountered in true_divide
  return (1 / (1 - (k3 * M - k1) / k2) - 1 ) ** (1/n)
<ipython-input-3-f0486a886956>:18: RuntimeWarning: invalid value encountered in power
  return (1 / (1 - (k3 * M - k1) / k2) - 1 ) ** (1/n)


In [4]:
# Create figure 8 from assignment

# def dMdt(M, A, L):
#     return M + A - 5

# def dAdt(M, A, L):
#     return 2 * M - 8 * A

# def M_nullcline(M, A, L):
#     return 5 - M

# def A_nullcline(M, A, L):
#     return 0.25 * M

In [5]:
%matplotlib widget

x_velocities = np.array(x_velocities)
y_velocities = np.array(y_velocities)

plt.ylim(As[0], As[-1])
plt.streamplot(Ms, As, x_velocities, y_velocities)

plt.plot(Ms, m_nulls, label="dm/dt = 0")
plt.plot(Ms, a_nulls_min, label="da/dt = 0")
plt.plot(Ms, a_nulls_plus, label="da/dt = 0")

plt.xlabel("M")
plt.ylabel("A")

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
def dMdt(M, A, L, Vmax=1, n=5, Km=2, k1=0.05, k2=1, k3=1, k4=1, k5=0.2):
    return k1 + k2 * (1 - (1/(1 + A**n))) - k3 * M

def dAdt(M, A, L, Vmax=1, n=5, Km=2, k1=0.05, k2=1, k3=1, k4=1, k5=0.2):
    return k4 * M * L - k5 * A - Vmax* M * A / (Km + A)

def M_nullcline(M, A, L, Vmax=1, n=5, Km=2, k1=0.05, k2=1, k3=1, k4=1, k5=0.2):

    return (k1 + k2 * (1 - 1/(1 + A**n))) / k3

def A_nullcline(M, A, L, Vmax=1, n=5, Km=2, k1=0.05, k2=1, k3=1, k4=1, k5=0.2):

    return k5 * A / (k4 * L - (Vmax * A) / (Km + A))


L = 1
As = np.arange(0, 3, 0.01)
Ms = np.arange(0, 2, 0.01)

x_velocities, y_velocities = [], []
for M in Ms:

    x_temp, y_temp = [], []
    
    for A in As:
        x_temp.append(dAdt(M, A, L))
        y_temp.append(dMdt(M, A, L))

    x_velocities.append(x_temp)
    y_velocities.append(y_temp)

m_nulls = M_nullcline(Ms, As, L)
a_nulls = A_nullcline(Ms, As, L)

In [7]:
%matplotlib widget

x_velocities = np.array(x_velocities)
y_velocities = np.array(y_velocities)

plt.ylim(Ms[0], Ms[-1])
plt.streamplot(As, Ms, x_velocities, y_velocities, density=1)

plt.plot(As, m_nulls, label="dm/dt = 0")
plt.plot(As, a_nulls, label="da/dt = 0")
# plt.plot(Ms, a_nulls_plus, label="da/dt = 0")

plt.xlabel("A")
plt.ylabel("M")

plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …